## Read and Clean Dataset


Import the libraries.


In [1]:
# Install tensorflow and keras in Anaconda
# !pip install tensorflow
# !pip install keras

In [2]:
import pandas as pd
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn import metrics

<strong>The dataset is about the compressive strength of different samples of concrete based on the volumes of the different ingredients that were used to make them. Ingredients include:</strong>

<strong>1. Cement</strong>

<strong>2. Blast Furnace Slag</strong>

<strong>3. Fly Ash</strong>

<strong>4. Water</strong>

<strong>5. Superplasticizer</strong>

<strong>6. Coarse Aggregate</strong>

<strong>7. Fine Aggregate</strong>


Read the data into a <em>pandas</em> dataframe.


In [3]:
concrete_data = pd.read_csv('concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


So the first concrete sample has 540 cubic meter of cement, 0 cubic meter of blast furnace slag, 0 cubic meter of fly ash, 162 cubic meter of water, 2.5 cubic meter of superplaticizer, 1040 cubic meter of coarse aggregate, 676 cubic meter of fine aggregate. Such a concrete mix which is 28 days old, has a compressive strength of 79.99 MPa.


#### Let's check how many data points we have.


In [4]:
concrete_data.shape

(1030, 9)

So, there are approximately 1000 samples to train our model on. Because of the few samples, we have to be careful not to overfit the training data.


Let's check the dataset for any missing values.


In [5]:
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [6]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

The data looks very clean and is ready to be used to build our model.


#### Split data into predictors (X) and target (y)


The target variable in this problem is the concrete sample strength. Therefore, our predictors will be all the other columns.


In [7]:
concrete_data_columns = concrete_data.columns

X = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
y = concrete_data['Strength'] # Strength column

<a id="item2"></a>


Let's do a quick sanity check of the predictors and the target dataframes.


In [8]:
X.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [9]:
y.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

### Finally, the last step is to normalize the data by substracting the mean and dividing by the standard deviation.

In [10]:
X_norm = (X - X.mean()) / X.std()
X_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


Let's save the number of predictors to *n_cols* since we will need this number when building our network.


In [11]:
n_cols = X_norm.shape[1] # number of predictors
print("Number of features:", n_cols)

Number of features: 8


<a id="item1"></a>


<a id='item32'></a>


## Build a Neural Network


Let's define a function that defines our neural network model for us so that we can conveniently call it to create our model.


In [12]:
# define regression model
def create_neural_network():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

The above function create a model that has one hidden layers, each of 10 hidden units.


<a id="item4"></a>


<a id='item34'></a>


## Train and Test the Network


Let's call the function now to create our model.


In [13]:
# build the model
model = create_neural_network()

Next, we will train and test the model at the same time using the *fit* method. We will leave out 30% of the data for validation and we will train the model for 100 epochs.


In [14]:
# fit the model
mean_sq_error = [] # Empty list of mean squared error

def split_and_fit_model(myModel):
    X_train, X_test, y_train, y_test = train_test_split( X_norm, y, test_size=0.3, random_state=4)
    
    myModel.fit(X_train, y_train, validation_split=0.3, epochs=50, verbose=2)
    
    # evaluate the model
    scores = myModel.evaluate(X_test, y_test, verbose=0)
    print("Accuracy: {} \n Error: {}".format(scores, 100-scores*100))
    
    # Get the predicted values
    yhat = myModel.predict(X_test)
    mse_sk = metrics.mean_squared_error(y_test, yhat) # Calculate RMSE
    mean_sq_error.append(mse_sk)

# Step 4
for i in range(50):
    print("Iteration %s:\n" %str(i))
    split_and_fit_model(model)
    
print(len(mean_sq_error))
print("Mean:", np.mean(np.array(mean_sq_error)))
print("Standard Deviation :", np.std(np.array(mean_sq_error)))

Iteration 0:

Epoch 1/50
16/16 - 1s - loss: 1659.3796 - val_loss: 1540.6627 - 659ms/epoch - 41ms/step
Epoch 2/50
16/16 - 0s - loss: 1640.9280 - val_loss: 1525.0547 - 36ms/epoch - 2ms/step
Epoch 3/50
16/16 - 0s - loss: 1626.9935 - val_loss: 1513.4404 - 34ms/epoch - 2ms/step
Epoch 4/50
16/16 - 0s - loss: 1616.4568 - val_loss: 1504.2495 - 35ms/epoch - 2ms/step
Epoch 5/50
16/16 - 0s - loss: 1607.9015 - val_loss: 1495.9752 - 37ms/epoch - 2ms/step
Epoch 6/50
16/16 - 0s - loss: 1599.0433 - val_loss: 1485.9482 - 44ms/epoch - 3ms/step
Epoch 7/50
16/16 - 0s - loss: 1586.6483 - val_loss: 1470.7136 - 35ms/epoch - 2ms/step
Epoch 8/50
16/16 - 0s - loss: 1567.0499 - val_loss: 1446.5380 - 37ms/epoch - 2ms/step
Epoch 9/50
16/16 - 0s - loss: 1537.8876 - val_loss: 1412.8303 - 37ms/epoch - 2ms/step
Epoch 10/50
16/16 - 0s - loss: 1499.1499 - val_loss: 1370.0850 - 37ms/epoch - 2ms/step
Epoch 11/50
16/16 - 0s - loss: 1449.7765 - val_loss: 1315.7837 - 46ms/epoch - 3ms/step
Epoch 12/50
16/16 - 0s - loss: 1387.

Epoch 46/50
16/16 - 0s - loss: 105.9765 - val_loss: 102.4189 - 36ms/epoch - 2ms/step
Epoch 47/50
16/16 - 0s - loss: 105.2426 - val_loss: 102.0074 - 38ms/epoch - 2ms/step
Epoch 48/50
16/16 - 0s - loss: 104.3638 - val_loss: 100.7910 - 36ms/epoch - 2ms/step
Epoch 49/50
16/16 - 0s - loss: 103.6123 - val_loss: 100.5327 - 36ms/epoch - 2ms/step
Epoch 50/50
16/16 - 0s - loss: 102.9985 - val_loss: 100.3577 - 37ms/epoch - 2ms/step
Accuracy: 131.61074829101562 
 Error: -13061.074829101562
10/10 [==============================] - 0s 886us/step
Iteration 2:

Epoch 1/50
16/16 - 0s - loss: 102.3157 - val_loss: 99.3541 - 70ms/epoch - 4ms/step
Epoch 2/50
16/16 - 0s - loss: 101.1853 - val_loss: 99.0054 - 34ms/epoch - 2ms/step
Epoch 3/50
16/16 - 0s - loss: 100.9217 - val_loss: 98.0509 - 37ms/epoch - 2ms/step
Epoch 4/50
16/16 - 0s - loss: 99.9758 - val_loss: 97.5416 - 39ms/epoch - 2ms/step
Epoch 5/50
16/16 - 0s - loss: 99.3142 - val_loss: 96.7178 - 36ms/epoch - 2ms/step
Epoch 6/50
16/16 - 0s - loss: 98.55

Epoch 42/50
16/16 - 0s - loss: 51.3666 - val_loss: 54.7549 - 35ms/epoch - 2ms/step
Epoch 43/50
16/16 - 0s - loss: 50.5207 - val_loss: 52.9565 - 36ms/epoch - 2ms/step
Epoch 44/50
16/16 - 0s - loss: 49.7692 - val_loss: 52.4104 - 36ms/epoch - 2ms/step
Epoch 45/50
16/16 - 0s - loss: 49.3137 - val_loss: 52.4121 - 36ms/epoch - 2ms/step
Epoch 46/50
16/16 - 0s - loss: 48.7913 - val_loss: 51.1964 - 35ms/epoch - 2ms/step
Epoch 47/50
16/16 - 0s - loss: 48.3876 - val_loss: 51.4035 - 34ms/epoch - 2ms/step
Epoch 48/50
16/16 - 0s - loss: 47.9036 - val_loss: 50.5237 - 37ms/epoch - 2ms/step
Epoch 49/50
16/16 - 0s - loss: 47.3621 - val_loss: 49.9179 - 36ms/epoch - 2ms/step
Epoch 50/50
16/16 - 0s - loss: 47.0299 - val_loss: 49.5029 - 34ms/epoch - 2ms/step
Accuracy: 56.21842575073242 
 Error: -5521.842575073242
10/10 [==============================] - 0s 994us/step
Iteration 4:

Epoch 1/50
16/16 - 0s - loss: 46.6289 - val_loss: 49.4584 - 71ms/epoch - 4ms/step
Epoch 2/50
16/16 - 0s - loss: 46.3906 - val_lo

Epoch 38/50
16/16 - 0s - loss: 33.2065 - val_loss: 44.2703 - 35ms/epoch - 2ms/step
Epoch 39/50
16/16 - 0s - loss: 33.4417 - val_loss: 44.5070 - 36ms/epoch - 2ms/step
Epoch 40/50
16/16 - 0s - loss: 33.1333 - val_loss: 43.3431 - 37ms/epoch - 2ms/step
Epoch 41/50
16/16 - 0s - loss: 33.1926 - val_loss: 43.8925 - 36ms/epoch - 2ms/step
Epoch 42/50
16/16 - 0s - loss: 33.1630 - val_loss: 43.8233 - 37ms/epoch - 2ms/step
Epoch 43/50
16/16 - 0s - loss: 33.2870 - val_loss: 43.9652 - 37ms/epoch - 2ms/step
Epoch 44/50
16/16 - 0s - loss: 33.0218 - val_loss: 43.9163 - 35ms/epoch - 2ms/step
Epoch 45/50
16/16 - 0s - loss: 32.8217 - val_loss: 43.9778 - 36ms/epoch - 2ms/step
Epoch 46/50
16/16 - 0s - loss: 32.7301 - val_loss: 44.0210 - 37ms/epoch - 2ms/step
Epoch 47/50
16/16 - 0s - loss: 32.6669 - val_loss: 43.8745 - 35ms/epoch - 2ms/step
Epoch 48/50
16/16 - 0s - loss: 32.8390 - val_loss: 43.8021 - 37ms/epoch - 2ms/step
Epoch 49/50
16/16 - 0s - loss: 32.6892 - val_loss: 44.6195 - 35ms/epoch - 2ms/step
Epoc

Epoch 34/50
16/16 - 0s - loss: 29.2802 - val_loss: 44.9389 - 35ms/epoch - 2ms/step
Epoch 35/50
16/16 - 0s - loss: 29.1047 - val_loss: 44.5128 - 34ms/epoch - 2ms/step
Epoch 36/50
16/16 - 0s - loss: 29.3160 - val_loss: 44.6538 - 35ms/epoch - 2ms/step
Epoch 37/50
16/16 - 0s - loss: 29.3159 - val_loss: 44.4923 - 37ms/epoch - 2ms/step
Epoch 38/50
16/16 - 0s - loss: 29.1086 - val_loss: 45.1756 - 35ms/epoch - 2ms/step
Epoch 39/50
16/16 - 0s - loss: 29.4696 - val_loss: 44.1209 - 36ms/epoch - 2ms/step
Epoch 40/50
16/16 - 0s - loss: 29.3432 - val_loss: 45.5019 - 36ms/epoch - 2ms/step
Epoch 41/50
16/16 - 0s - loss: 29.0510 - val_loss: 44.2946 - 36ms/epoch - 2ms/step
Epoch 42/50
16/16 - 0s - loss: 29.0906 - val_loss: 45.0576 - 34ms/epoch - 2ms/step
Epoch 43/50
16/16 - 0s - loss: 29.1151 - val_loss: 44.8007 - 37ms/epoch - 2ms/step
Epoch 44/50
16/16 - 0s - loss: 29.4479 - val_loss: 43.8414 - 36ms/epoch - 2ms/step
Epoch 45/50
16/16 - 0s - loss: 29.4034 - val_loss: 45.4209 - 38ms/epoch - 2ms/step
Epoc

...

Epoch 14/50
16/16 - 0s - loss: 15.5958 - val_loss: 37.4084 - 40ms/epoch - 3ms/step
Epoch 15/50
16/16 - 0s - loss: 15.3706 - val_loss: 38.9961 - 41ms/epoch - 3ms/step
Epoch 16/50
16/16 - 0s - loss: 15.6789 - val_loss: 37.2316 - 46ms/epoch - 3ms/step
Epoch 17/50
16/16 - 0s - loss: 15.5584 - val_loss: 38.3583 - 39ms/epoch - 2ms/step
Epoch 18/50
16/16 - 0s - loss: 15.2803 - val_loss: 37.7388 - 37ms/epoch - 2ms/step
Epoch 19/50
16/16 - 0s - loss: 15.4673 - val_loss: 37.2635 - 40ms/epoch - 3ms/step
Epoch 20/50
16/16 - 0s - loss: 15.3545 - val_loss: 37.9405 - 36ms/epoch - 2ms/step
Epoch 21/50
16/16 - 0s - loss: 15.5472 - val_loss: 38.0165 - 36ms/epoch - 2ms/step
Epoch 22/50
16/16 - 0s - loss: 15.3559 - val_loss: 38.1086 - 34ms/epoch - 2ms/step
Epoch 23/50
16/16 - 0s - loss: 15.3689 - val_loss: 38.6409 - 38ms/epoch - 2ms/step
Epoch 24/50
16/16 - 0s - loss: 15.2599 - val_loss: 37.3833 - 36ms/epoch - 2ms/step
Epoch 25/50
16/16 - 0s - loss: 15.5356 - val_loss: 39.9725 - 38ms/epoch - 2ms/step
Epoc

Epoch 10/50
16/16 - 0s - loss: 15.3961 - val_loss: 37.7445 - 37ms/epoch - 2ms/step
Epoch 11/50
16/16 - 0s - loss: 15.2522 - val_loss: 37.7006 - 36ms/epoch - 2ms/step
Epoch 12/50
16/16 - 0s - loss: 15.4258 - val_loss: 37.2992 - 35ms/epoch - 2ms/step
Epoch 13/50
16/16 - 0s - loss: 15.2827 - val_loss: 39.1688 - 33ms/epoch - 2ms/step
Epoch 14/50
16/16 - 0s - loss: 15.2295 - val_loss: 37.4862 - 36ms/epoch - 2ms/step
Epoch 15/50
16/16 - 0s - loss: 15.1404 - val_loss: 37.4701 - 34ms/epoch - 2ms/step
Epoch 16/50
16/16 - 0s - loss: 15.3712 - val_loss: 39.2227 - 37ms/epoch - 2ms/step
Epoch 17/50
16/16 - 0s - loss: 15.4776 - val_loss: 37.0210 - 36ms/epoch - 2ms/step
Epoch 18/50
16/16 - 0s - loss: 15.3859 - val_loss: 38.6787 - 36ms/epoch - 2ms/step
Epoch 19/50
16/16 - 0s - loss: 15.4240 - val_loss: 38.3740 - 37ms/epoch - 2ms/step
Epoch 20/50
16/16 - 0s - loss: 15.4608 - val_loss: 38.7314 - 35ms/epoch - 2ms/step
Epoch 21/50
16/16 - 0s - loss: 15.5660 - val_loss: 38.2420 - 35ms/epoch - 2ms/step
Epoc

Epoch 6/50
16/16 - 0s - loss: 15.2353 - val_loss: 37.9222 - 38ms/epoch - 2ms/step
Epoch 7/50
16/16 - 0s - loss: 15.2604 - val_loss: 38.0036 - 37ms/epoch - 2ms/step
Epoch 8/50
16/16 - 0s - loss: 15.3829 - val_loss: 37.2997 - 35ms/epoch - 2ms/step
Epoch 9/50
16/16 - 0s - loss: 15.2852 - val_loss: 38.0968 - 36ms/epoch - 2ms/step
Epoch 10/50
16/16 - 0s - loss: 15.0618 - val_loss: 37.5378 - 33ms/epoch - 2ms/step
Epoch 11/50
16/16 - 0s - loss: 15.2864 - val_loss: 38.0318 - 35ms/epoch - 2ms/step
Epoch 12/50
16/16 - 0s - loss: 15.3117 - val_loss: 37.9969 - 33ms/epoch - 2ms/step
Epoch 13/50
16/16 - 0s - loss: 15.1648 - val_loss: 37.2028 - 34ms/epoch - 2ms/step
Epoch 14/50
16/16 - 0s - loss: 15.2454 - val_loss: 38.9716 - 36ms/epoch - 2ms/step
Epoch 15/50
16/16 - 0s - loss: 15.0633 - val_loss: 36.9421 - 35ms/epoch - 2ms/step
Epoch 16/50
16/16 - 0s - loss: 15.7392 - val_loss: 37.6086 - 35ms/epoch - 2ms/step
Epoch 17/50
16/16 - 0s - loss: 15.4833 - val_loss: 38.5249 - 35ms/epoch - 2ms/step
Epoch 18

Epoch 2/50
16/16 - 0s - loss: 14.7907 - val_loss: 39.1196 - 38ms/epoch - 2ms/step
Epoch 3/50
16/16 - 0s - loss: 15.0759 - val_loss: 37.2093 - 37ms/epoch - 2ms/step
Epoch 4/50
16/16 - 0s - loss: 15.0358 - val_loss: 39.5156 - 37ms/epoch - 2ms/step
Epoch 5/50
16/16 - 0s - loss: 15.2429 - val_loss: 37.3820 - 37ms/epoch - 2ms/step
Epoch 6/50
16/16 - 0s - loss: 15.0434 - val_loss: 38.2243 - 37ms/epoch - 2ms/step
Epoch 7/50
16/16 - 0s - loss: 15.2383 - val_loss: 37.3705 - 38ms/epoch - 2ms/step
Epoch 8/50
16/16 - 0s - loss: 15.1027 - val_loss: 37.9809 - 36ms/epoch - 2ms/step
Epoch 9/50
16/16 - 0s - loss: 15.1277 - val_loss: 37.4154 - 36ms/epoch - 2ms/step
Epoch 10/50
16/16 - 0s - loss: 14.9477 - val_loss: 38.6149 - 37ms/epoch - 2ms/step
Epoch 11/50
16/16 - 0s - loss: 15.2318 - val_loss: 37.5725 - 37ms/epoch - 2ms/step
Epoch 12/50
16/16 - 0s - loss: 15.0427 - val_loss: 37.2390 - 38ms/epoch - 2ms/step
Epoch 13/50
16/16 - 0s - loss: 15.0623 - val_loss: 38.5032 - 36ms/epoch - 2ms/step
Epoch 14/50


Epoch 50/50
16/16 - 0s - loss: 14.8434 - val_loss: 37.1879 - 38ms/epoch - 2ms/step
Accuracy: 37.66233444213867 
 Error: -3666.233444213867
10/10 [==============================] - 0s 997us/step
50
Mean: 45.330837766071134
Standard Deviation : 23.34849308478311


<a id="item4"></a>

<a id="item4"></a>

The mean from part B: 63.55776134625607

The mean from part D: 45.330837766071134

The mean in step D is less than in B